<a href="https://colab.research.google.com/github/Kovibordy/ML-2.0/blob/main/11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Лабораторная работа 2

Вариант 11

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.linear_model import LinearRegression

# Загрузка данных
df = pd.read_csv('v11.csv')
print("Загружено строк:", df.shape[0], "столбцов:", df.shape[1])
print("\nПервые 5 строк:")
df.head()

Загружено строк: 30 столбцов: 20

Первые 5 строк:


,ID_дома,Адрес,Тип_дома,Площадь_м2,Год_постройки,Количество_жильцов,Температура_на_улице_C,Температура_в_доме_C,Потребление_электричества_кВтч,Потребление_воды_м3,Потребление_газа_м3,Количество_окон,Толщина_утепления_см,Наличие_солнечных_панелей,Выработка_солнечной_энергии_кВтч,Система_умного_отопления,Эффективность_отопления_%,Уровень_автоматизации,Стоимость_коммуналки_руб,Энергоэффективность_класс
0,1,"ул. Ленина, 15",Квартира,65,2010,2,18.5,21.0,280.0,8.5,45.0,4,10.0,Нет,0.0,Нет,85.0,3,4500.0,A
1,2,"пр. Мира, 28",Частный дом,120,2005,4,20.2,22.0,450.0,15.2,120.0,8,15.0,Да,85.0,Да,92.0,8,8500.0,B
2,3,"ул. Садовая, 7",Таунхаус,85,2015,3,17.8,20.5,320.0,10.8,NaN,6,12.0,Нет,0.0,Да,88.0,6,5200.0,C
3,4,"ул. Центральная, 42",Квартира,75,2018,2,22.1,23.0,310.0,9.3,52.0,5,8.0,Да,65.0,Нет,82.0,4,4800.0,D
4,5,"пер. Зеленый, 3",Частный дом,150,1995,5,16.3,19.5,NaN,20.5,180.0,10,20.0,Нет,0.0,Да,78.0,7,12500.0,E


In [16]:
print("ВИЗУАЛЬНЫЙ АНАЛИЗ\n")

# 1. Основная информация
print("1. Информация о данных:")
df.info()

# 2. Пропущенные значения
print("\n2. Пропущенные значения:")
missing = df.isnull().sum()
print(missing[missing > 0])

# 3. Категориальные признаки
print("\n3. Категориальные признаки:")
cat_cols = df.select_dtypes(include=['object']).columns
for col in cat_cols:
    if col != 'Адрес':
        print(f"{col}: {df[col].nunique()} уникальных значений")

ВИЗУАЛЬНЫЙ АНАЛИЗ

1. Информация о данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID_дома                           30 non-null     int64  
 1   Адрес                             30 non-null     object 
 2   Тип_дома                          28 non-null     object 
 3   Площадь_м2                        30 non-null     int64  
 4   Год_постройки                     30 non-null     int64  
 5   Количество_жильцов                30 non-null     int64  
 6   Температура_на_улице_C            30 non-null     float64
 7   Температура_в_доме_C              30 non-null     float64
 8   Потребление_электричества_кВтч    27 non-null     float64
 9   Потребление_воды_м3               28 non-null     float64
 10  Потребление_газа_м3               28 non-null     float64
 11  Количество_окон               

Видим 8 столбцов с пропусками, наиболее проблемные - потребление электричества и толщина утепления (по 3 пропуска). Имеются 4 категориальных признака для кодирования.

In [17]:
#Кодирование категориальных признаков
print("КОДИРОВАНИЕ ПРИЗНАКОВ\n")

# Сохраняем копию
df_original = df.copy()

# 1. Бинарные признаки (Да/Нет -> 1/0)
binary_cols = ['Наличие_солнечных_панелей', 'Система_умного_отопления']
for col in binary_cols:
    df[col] = df[col].map({'Да': 1, 'Нет': 0})
    print(f"Закодирован '{col}': Да→1, Нет→0")

# 2. One-Hot Encoding для "Тип_дома"
type_dummies = pd.get_dummies(df['Тип_дома'], prefix='Тип')
df = pd.concat([df, type_dummies], axis=1)
df = df.drop('Тип_дома', axis=1)
print(f"One-Hot Encoding для 'Тип_дома': создано {type_dummies.shape[1]} новых столбцов")

# 3. Ordinal Encoding для "Энергоэффективность_класс" (A=0, B=1, C=2, D=3, E=4)
energy_map = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
df['Энергоэффективность_класс'] = df['Энергоэффективность_класс'].map(energy_map)
print(f"Ordinal Encoding для 'Энергоэффективность_класс': A→0, B→1, C→2, D→3, E→4")

print("\nДанные после кодирования:")
print(df.head())

КОДИРОВАНИЕ ПРИЗНАКОВ

Закодирован 'Наличие_солнечных_панелей': Да→1, Нет→0
Закодирован 'Система_умного_отопления': Да→1, Нет→0
One-Hot Encoding для 'Тип_дома': создано 3 новых столбцов
Ordinal Encoding для 'Энергоэффективность_класс': A→0, B→1, C→2, D→3, E→4

Данные после кодирования:
   ID_дома                Адрес  Площадь_м2  Год_постройки  \
0        1       ул. Ленина, 15          65           2010   
1        2         пр. Мира, 28         120           2005   
2        3       ул. Садовая, 7          85           2015   
3        4  ул. Центральная, 42          75           2018   
4        5      пер. Зеленый, 3         150           1995   

   Количество_жильцов  Температура_на_улице_C  Температура_в_доме_C  \
0                   2                    18.5                  21.0   
1                   4                    20.2                  22.0   
2                   3                    17.8                  20.5   
3                   2                    22.1           

In [18]:
#Устранение пропусков - Метод 1 (статистические характеристики)
print("МЕТОД 1: СТАТИСТИЧЕСКИЕ ХАРАКТЕРИСТИКИ\n")

df_stats = df.copy()

# Заполняем числовые столбцы медианой
numeric_cols = df_stats.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    if df_stats[col].isnull().any():
        median_val = df_stats[col].median()
        na_count = df_stats[col].isnull().sum()
        df_stats[col] = df_stats[col].fillna(median_val)
        print(f"Заполнено '{col}': {na_count} пропусков → медиана={median_val:.2f}")

print(f"\nПосле метода 1 осталось пропусков: {df_stats.isnull().sum().sum()}")

МЕТОД 1: СТАТИСТИЧЕСКИЕ ХАРАКТЕРИСТИКИ

Заполнено 'Потребление_электричества_кВтч': 3 пропусков → медиана=370.00
Заполнено 'Потребление_воды_м3': 2 пропусков → медиана=12.00
Заполнено 'Потребление_газа_м3': 2 пропусков → медиана=80.00
Заполнено 'Толщина_утепления_см': 3 пропусков → медиана=10.00
Заполнено 'Выработка_солнечной_энергии_кВтч': 3 пропусков → медиана=0.00
Заполнено 'Эффективность_отопления_%': 2 пропусков → медиана=84.50
Заполнено 'Стоимость_коммуналки_руб': 2 пропусков → медиана=6100.00

После метода 1 осталось пропусков: 0


Метод статистических характеристик заполнил все пропуски медианными значениями. Самые значительные заполнения: потребление электричества 370 кВтч, стоимость коммуналки 6100 руб

In [19]:
#Устранение пропусков - Метод 2 (K ближайших соседей)
print("МЕТОД 2: K БЛИЖАЙШИХ СОСЕДЕЙ\n")

df_knn = df.copy()

# Выбираем числовые столбцы
numeric_cols = df_knn.select_dtypes(include=[np.number]).columns.tolist()

# Применяем KNNImputer
knn_imputer = KNNImputer(n_neighbors=3)
df_knn[numeric_cols] = knn_imputer.fit_transform(df_knn[numeric_cols])

print(f"KNNImputer с n_neighbors=3 применен к {len(numeric_cols)} числовым признакам")
print(f"После метода 2 осталось пропусков: {df_knn.isnull().sum().sum()}")

МЕТОД 2: K БЛИЖАЙШИХ СОСЕДЕЙ

KNNImputer с n_neighbors=3 применен к 18 числовым признакам
После метода 2 осталось пропусков: 0


KNN метод с 3 соседями успешно устранил все пропуски, используя сходство между объектами. Метод обработал 18 числовых признаков

In [20]:
#Устранение пропусков - Метод 3 (линейная регрессия)
print("МЕТОД 3: ЛИНЕЙНАЯ РЕГРЕССИЯ\n")

df_lr = df.copy()

# Берем столбец с пропусками для примера (Потребление_электричества_кВтч)
target_col = 'Потребление_электричества_кВтч'

if df_lr[target_col].isnull().any():
    # Разделяем на данные с известным и неизвестным target
    train_data = df_lr[df_lr[target_col].notna()]
    predict_data = df_lr[df_lr[target_col].isna()]

    # Выбираем признаки для регрессии
    feature_cols = ['Площадь_м2', 'Количество_жильцов', 'Температура_в_доме_C']

    # Убираем строки с пропусками в признаках
    train_data = train_data.dropna(subset=feature_cols)
    predict_data = predict_data.dropna(subset=feature_cols)

    if len(train_data) > 5 and len(predict_data) > 0:
        # Обучаем модель
        X_train = train_data[feature_cols]
        y_train = train_data[target_col]

        model = LinearRegression()
        model.fit(X_train, y_train)

        # Предсказываем
        X_predict = predict_data[feature_cols]
        predictions = model.predict(X_predict)

        # Заполняем пропуски
        df_lr.loc[predict_data.index, target_col] = predictions

        print(f"Обучена линейная регрессия для '{target_col}'")
        print(f"R² на обучающей выборке: {model.score(X_train, y_train):.3f}")
        print(f"Заполнено {len(predictions)} пропусков")
    else:
        print(f"Недостаточно данных для регрессии по '{target_col}'")
        # Заполняем медианой как запасной вариант
        median_val = df_lr[target_col].median()
        df_lr[target_col] = df_lr[target_col].fillna(median_val)
        print(f"Заполнено медианой: {median_val:.2f}")
else:
    print(f"В столбце '{target_col}' нет пропусков")

print(f"\nПосле метода 3 осталось пропусков: {df_lr.isnull().sum().sum()}")

МЕТОД 3: ЛИНЕЙНАЯ РЕГРЕССИЯ

Обучена линейная регрессия для 'Потребление_электричества_кВтч'
R² на обучающей выборке: 0.949
Заполнено 3 пропусков

После метода 3 осталось пропусков: 14


Линейная регрессия показала высокое качество для прогноза потребления электричества, но заполнила только целевой столбец, оставив 14 пропусков в других признаках

In [23]:
#Сравнение методов и итоги
print("СРАВНЕНИЕ МЕТОДОВ И ИТОГИ\n")

print("Результаты применения методов:")
print("-" * 50)
print(f"Метод 1 (Статистика): {df_stats.isnull().sum().sum()} пропусков осталось")
print(f"Метод 2 (KNN): {df_knn.isnull().sum().sum()} пропусков осталось")
print(f"Метод 3 (Регрессия): {df_lr.isnull().sum().sum()} пропусков осталось")

# Пример сравнения значений для одного столбца
example_col = 'Потребление_электричества_кВтч'
if example_col in df.columns:
    print(f"\nСравнение для столбца '{example_col}':")
    print(f"Исходное среднее: {df[example_col].mean():.2f}")
    print(f"Метод 1 (медиана): {df_stats[example_col].mean():.2f}")
    print(f"Метод 2 (KNN): {df_knn[example_col].mean():.2f}")
    print(f"Метод 3 (регрессия): {df_lr[example_col].mean():.2f}")

СРАВНЕНИЕ МЕТОДОВ И ИТОГИ

Результаты применения методов:
--------------------------------------------------
Метод 1 (Статистика): 0 пропусков осталось
Метод 2 (KNN): 0 пропусков осталось
Метод 3 (Регрессия): 14 пропусков осталось

Сравнение для столбца 'Потребление_электричества_кВтч':
Исходное среднее: 399.26
Метод 1 (медиана): 396.33
Метод 2 (KNN): 411.56
Метод 3 (регрессия): 403.62


Сравнение методов показало, что KNN дает наиболее полное устранение пропусков, а регрессия - наиболее точное для конкретного признака. Средние значения после обработки различаются незначительно.